In [1]:
import pandas as pd
import datetime
import numpy
import re
from sklearn.naive_bayes import GaussianNB
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Function for calculating age of a person given his date of birth
from datetime import date
def calculate_age(born):
    today = date.today()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

In [3]:
#Function for calculating number of days between two dates 
def datediff(any_date):
    today = date.today()
    datediff = today-any_date 
    return datediff.days

In [4]:
# Calculating the Tenure in months given the data is in X Years and Y months format
def age_length(TenureAsYearsAndMonths):
    years = int(re.findall(r'(\d+)yrs',TenureAsYearsAndMonths )[0])
    months = int(re.findall(r'(\d+)mon',TenureAsYearsAndMonths )[0])
    total_tenure = (years*12)+months
    return total_tenure

In [5]:
# Function for doing all the data manipulations
def AllDataManipulations(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['loan_default','branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = pd.get_dummies(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],prefix='Bureau_score_description')
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES',
       'loan_default']],temp1,temp2],axis=1)
    return Manip_Data,Manip_Completed

In [6]:
MasterData = pd.read_csv("train.csv")
Data1 = MasterData.copy()

In [7]:
Train_ManipData_1,Train_ManipCompleted_1 = AllDataManipulations(Data1)

In [8]:
Train_ManipCompleted_1.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',

In [9]:
IndeVectors = Train_ManipCompleted_1[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available']]
Labels = Train_ManipCompleted_1['loan_default']

In [10]:
train_x, valid_x, train_y, valid_y = model_selection.train_test_split(IndeVectors,Labels,random_state = 1234,test_size = 0.10)

# Random forest
rf_model_1 = RandomForestClassifier(n_estimators=300,bootstrap = True,max_features = 'sqrt')
rf_model_1.fit(train_x,list(train_y))
ypred_rf1 = rf_model_1.predict(valid_x)
auc_rf1 = metrics.roc_auc_score(valid_y,ypred_rf1)
print("AUC of Random forest Model_1: ",auc_rf1)
# Naive Bayes
gnb = GaussianNB()
gnb.fit(train_x,train_y)
ypred_gnb = gnb.predict(valid_x)
auc_gnb = metrics.roc_auc_score(valid_y,ypred_gnb)
print("AUC of Naive Bayes Model_1: ",auc_gnb)

In [11]:
# Modifying data manipulations function to work for test data as it doesn't have labels column
# Function for doing all the data manipulations
def AllDataManipulations_forTest(MainDataFrame,Manip_Data = pd.DataFrame,temp1 = pd.DataFrame, temp2 = pd.DataFrame,Manip_Completed = pd.DataFrame):
    Manip_Data = MainDataFrame.copy()
    Manip_Data['Employment.Type'].fillna(Manip_Data['Employment.Type'].mode()[0],inplace = True)
    Manip_Data[['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']] = Manip_Data[
        ['branch_id','supplier_id','manufacturer_id','Current_pincode_ID','State_ID','Employee_code_ID']].apply(lambda x: x.astype('category'))
    Manip_Data['Date.of.Birth'] = pd.to_datetime(Manip_Data['Date.of.Birth'])
    Manip_Data['Age'] = Manip_Data['Date.of.Birth'].apply(lambda x: calculate_age(x))
    Manip_Data['DisbursalDate'] = pd.to_datetime(Manip_Data['DisbursalDate'])
    Manip_Data['HowManyDaysSinceDisburse'] = Manip_Data['DisbursalDate'].dt.date.apply(lambda x: datediff(x))
    Manip_Data['AvgAcctAgeInMonths'] = Manip_Data['AVERAGE.ACCT.AGE'].apply(lambda x: age_length(x))
    Manip_Data['CredHistLenInMonts'] = Manip_Data['CREDIT.HISTORY.LENGTH'].apply(lambda x: age_length(x))
    temp1 = pd.get_dummies(Manip_Data['Employment.Type'],prefix='EmploymentType')
    temp2 = pd.get_dummies(Manip_Data['PERFORM_CNS.SCORE.DESCRIPTION'],prefix='Bureau_score_description')
    Manip_Completed = pd.concat([Manip_Data[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id',
       'supplier_id', 'manufacturer_id', 'Current_pincode_ID','Age',
       'HowManyDaysSinceDisburse','State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE',
       'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 'PRI.SANCTIONED.AMOUNT',
       'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
       'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
       'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
       'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS',
       'AvgAcctAgeInMonths','CredHistLenInMonts','NO.OF_INQUIRIES']],temp1,temp2],axis=1)
    return Manip_Data,Manip_Completed

In [12]:
Master_test = pd.read_csv("test_bqCt9Pv.csv")
Test_Data = Master_test.copy()
Test_ManipData,Test_ManipCompleted = AllDataManipulations_forTest(Test_Data)

In [13]:
Test_Data.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES
0,655269,53478,63558,86.54,67,22807,45,1497,01-01-74,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
1,723482,55513,63163,89.45,67,22807,45,1497,20-05-85,Self employed,...,0,0,0,5605,0,1,0,0yrs 8mon,1yrs 0mon,1
2,758529,65282,84320,79.93,78,23135,86,2071,14-10-95,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0
3,763449,46905,63896,76.58,78,17014,45,2070,01-06-73,Self employed,...,0,0,0,0,0,0,0,2yrs 5mon,2yrs 5mon,0
4,708663,51428,63896,86.08,78,17014,45,2069,01-06-72,Salaried,...,0,0,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0


In [14]:
Test_ManipCompleted.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'EmploymentType_Salaried',
       'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_

In [15]:
Test_ManipCompleted['Bureau_score_description_Not Scored: More than 50 active Accounts found'] = 0

ActualTestPred_rf1 = rf_model_1.predict(Test_ManipCompleted)
ActualTestPred_nb = gnb.predict(Test_ManipCompleted)
RF1_Submissions = pd.concat([Test_Data['UniqueID'],pd.DataFrame(ActualTestPred_rf1,columns=['loan_default'])],axis=1)
nb_Submissions = pd.concat([Test_Data['UniqueID'],pd.DataFrame(ActualTestPred_nb,columns=['loan_default'])],axis=1)

RF1_Submissions.to_csv('RF1_Submissions.csv',index=False)

# Finding Feature importance
RF_feature_importances = pd.DataFrame(rf_model_1.feature_importances_,
                                   index = train_x.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)

In [20]:
# Building a separate dataframe with branch details to exploratory analysis
Branch_Master_Data = Data1[['loan_default', 'disbursed_amount', 'branch_id', 'Current_pincode_ID','DisbursalDate','State_ID',
                            'PRI.NO.OF.ACCTS', 'PRI.ACTIVE.ACCTS','PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE', 
                            'PRI.SANCTIONED.AMOUNT','PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS', 'SEC.ACTIVE.ACCTS',
                           'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE', 'SEC.SANCTIONED.AMOUNT',
                           'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT', 'SEC.INSTAL.AMT',
                            'NEW.ACCTS.IN.LAST.SIX.MONTHS', 'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']]

In [21]:
Branch_Master_Data.head()

,loan_default,disbursed_amount,branch_id,Current_pincode_ID,DisbursalDate,State_ID,PRI.NO.OF.ACCTS,PRI.ACTIVE.ACCTS,PRI.OVERDUE.ACCTS,PRI.CURRENT.BALANCE,...,SEC.NO.OF.ACCTS,SEC.ACTIVE.ACCTS,SEC.OVERDUE.ACCTS,SEC.CURRENT.BALANCE,SEC.SANCTIONED.AMOUNT,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS
0,0,50578,67,1441,03-08-18,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,47145,67,1502,26-09-18,6,1,1,1,27600,...,0,0,0,0,0,0,1991,0,0,1
2,0,53278,67,1497,01-08-18,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,57513,67,1501,26-10-18,6,3,0,0,0,...,0,0,0,0,0,0,31,0,0,0
4,1,52378,67,1495,26-09-18,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
Branch_Lookup = pd.DataFrame()
#type(Branch_Loop)
#Branch_Lookup['Branch_ID'] = list(Branch_Master_Data['branch_id'].unique())

In [56]:
Branch_Lookup.head()

""


In [158]:
# Function to find NPA% of a given branch, NPA% is being calculated by No.of defaulted out of Total No.of Loans given
def NPAFinder(TotalDataFrame,tempDf_1=pd.DataFrame(),tempDf_2 =pd.DataFrame(),tempDf_3 = pd.DataFrame()):
    tempDf_1 = pd.DataFrame(TotalDataFrame['branch_id'].value_counts())
    tempDf_1.reset_index(level=0, inplace=True)
    tempDf_1.columns = ['Branch_ID','TotalNoOfLoansDisbursed']
    tempDf_2 = pd.DataFrame(TotalDataFrame.groupby('branch_id')['loan_default'].value_counts())
    tempDf_2.reset_index(level=0, inplace=True)
    tempDf_2.columns = ['branch_id ','NoOfLoansDefaulted']
    tempDf_2.reset_index(level=0, inplace=True)
    tempDf_2 = tempDf_2[(tempDf_2['loan_default'] == 1)]
    tempDf3 = pd.merge(tempDf_2, tempDf_1, how='right', left_on= 'branch_id ', right_on= 'Branch_ID',
         left_index=False, right_index=False)
    tempDf3 = tempDf3[['Branch_ID','TotalNoOfLoansDisbursed','NoOfLoansDefaulted']]
    tempDf3['NPA%'] = ((tempDf3['NoOfLoansDefaulted'])/(tempDf3['TotalNoOfLoansDisbursed']))*100
    return tempDf3

In [171]:
NPA_df = NPAFinder(Branch_Master_Data)

In [172]:
NPA_df.head()

,Branch_ID,TotalNoOfLoansDisbursed,NoOfLoansDefaulted,NPA%
0,1,5709,853,14.941321
1,2,13138,2455,18.686254
2,3,9230,1614,17.486457
3,5,9218,2047,22.206552
4,7,3222,656,20.360025


In [173]:
NPA_df['Branch_ID'] = NPA_df['Branch_ID'].astype('category')

In [174]:
print(type(Train_ManipCompleted_1))
print(type(NPA_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [175]:
Train_ManipCompleted_2 = pd.merge(Train_ManipCompleted_1,NPA_df,how ='left', left_on='branch_id',right_on='Branch_ID')

In [176]:
Train_ManipCompleted_2.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',

In [177]:
Train_ManipCompleted_2.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Age,HowManyDaysSinceDisburse,State_ID,...,Bureau_score_description_Not Scored: More than 50 active Accounts found,Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive),Bureau_score_description_Not Scored: No Updates available in last 36 months,Bureau_score_description_Not Scored: Not Enough Info available on the customer,Bureau_score_description_Not Scored: Only a Guarantor,Bureau_score_description_Not Scored: Sufficient History Not Available,Branch_ID,TotalNoOfLoansDisbursed,NoOfLoansDefaulted,NPA%
0,50578,58400,89.55,67,22807,45,1441,35,403,6,...,0,0,0,0,0,0,67,11328,2198,19.403249
1,47145,65550,73.23,67,22807,45,1502,33,201,6,...,0,0,0,0,0,0,67,11328,2198,19.403249
2,53278,61360,89.63,67,22807,45,1497,33,462,6,...,0,0,0,0,0,0,67,11328,2198,19.403249
3,57513,66113,88.48,67,22807,45,1501,25,171,6,...,0,0,0,0,0,0,67,11328,2198,19.403249
4,52378,60300,88.39,67,22807,45,1495,41,201,6,...,0,0,0,0,0,0,67,11328,2198,19.403249


In [179]:
Train_ManipCompleted_2 = Train_ManipCompleted_2[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available','NPA%']]

In [181]:
Train_ManipCompleted_2.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Age,HowManyDaysSinceDisburse,State_ID,...,Bureau_score_description_L-Very High Risk,Bureau_score_description_M-Very High Risk,Bureau_score_description_No Bureau History Available,Bureau_score_description_Not Scored: More than 50 active Accounts found,Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive),Bureau_score_description_Not Scored: No Updates available in last 36 months,Bureau_score_description_Not Scored: Not Enough Info available on the customer,Bureau_score_description_Not Scored: Only a Guarantor,Bureau_score_description_Not Scored: Sufficient History Not Available,NPA%
0,50578,58400,89.55,67,22807,45,1441,35,403,6,...,0,0,1,0,0,0,0,0,0,19.403249
1,47145,65550,73.23,67,22807,45,1502,33,201,6,...,0,0,0,0,0,0,0,0,0,19.403249
2,53278,61360,89.63,67,22807,45,1497,33,462,6,...,0,0,1,0,0,0,0,0,0,19.403249
3,57513,66113,88.48,67,22807,45,1501,25,171,6,...,1,0,0,0,0,0,0,0,0,19.403249
4,52378,60300,88.39,67,22807,45,1495,41,201,6,...,0,0,1,0,0,0,0,0,0,19.403249


In [182]:
IndeVectors_2 = Train_ManipCompleted_2[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available','NPA%']]
Labels_2 = Train_ManipCompleted_1['loan_default']

In [183]:
train_x,valid_x,train_y,valid_y = model_selection.train_test_split(IndeVectors_2,Labels_2,random_state = 1234, test_size = 0.10)

In [184]:
len(train_x.columns)

60

In [186]:
rf_fe_1 = RandomForestClassifier(n_estimators=300, bootstrap=False,max_features=60)
rf_fe_1.fit(train_x,list(train_y))

RandomForestClassifier(bootstrap=False, class_weight=None, criterion='gini',
            max_depth=None, max_features=60, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [188]:
ypred_rf_fe_1 = rf_fe_1.predict(valid_x)
#print("Area Under Curve for Random forest first model after a feature Engg: ",metrics.roc_auc_score(valid_y,ypred_rf_fe_1))

In [190]:
metrics.roc_auc_score(list(valid_y),ypred_rf_fe_1)

0.5308318888405792

In [192]:
list(train_y)

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,


In [195]:
Train_ManipCompleted_1['supplier_id'].unique()

[22807, 22744, 17014, 20700, 15196, ..., 23389, 23575, 23618, 22289, 24031]
Length: 2953
Categories (2953, int64): [22807, 22744, 17014, 20700, ..., 23575, 23618, 22289, 24031]

In [278]:
# Function to % defaulted for each supplier
def Supplier_DefaulterFinder(TotalDataFrame,tempDf_1=pd.DataFrame(),tempDf_2 =pd.DataFrame(),tempDf_3 = pd.DataFrame()):
    tempDf_1 = pd.DataFrame(TotalDataFrame['supplier_id'].value_counts())
    tempDf_1.reset_index(level=0, inplace=True)
    tempDf_1.columns = ['supplier_id_1','TotalNoOfVehiclesSold']
    tempDf_2 = pd.DataFrame(TotalDataFrame.groupby('supplier_id')['loan_default'].value_counts())
    tempDf_2.reset_index(level=0, inplace=True)
    tempDf_2.columns = ['supplier_id_2','NoOfDefaulted_Sales']
    tempDf_2.reset_index(level=0, inplace=True)
    tempDf_2 = tempDf_2[(tempDf_2['loan_default'] == 1)]
    tempDf_1['supplier_id_1'] = tempDf_1['supplier_id_1'].astype('int')
    tempDf_2['supplier_id_2'] = tempDf_2['supplier_id_2'].astype('int')
    tempDf3 = pd.merge(tempDf_2, tempDf_1, how='right', on= ['supplier_id_2','supplier_id_1'],
                       left_index=False, right_index=False)
    tempDf3 = tempDf3[['supplier_id','TotalNoOfVehiclesSold','NoOfDefaulted_Sales']]
    tempDf3['Defaults%_DueToSupplier'] = ((tempDf3['NoOfDefaulted_Sales'])/(tempDf3['TotalNoOfVehiclesSold']))*100
    return tempDf3

In [279]:
Supplier_Defaulter_DF = Supplier_DefaulterFinder(Train_ManipCompleted_1)

KeyError: 'supplier_id_2'

In [203]:
Supplier_Defaulter_DF.head()

,supplier_id,TotalNoOfVehiclesSold,NoOfDefaulted_Sales,Defaults%_DueToSupplier
0,10524,6,3.0,50.000000
1,12312,46,5.0,10.869565
2,12374,99,14.0,14.141414
3,12441,47,10.0,21.276596
4,12456,72,16.0,22.222222


In [208]:
Supplier_Defaulter_DF.columns

Index(['supplier_id', 'TotalNoOfVehiclesSold', 'NoOfDefaulted_Sales',
       'Defaults%_DueToSupplier'],
      dtype='object')

In [217]:
Supplier_Defaulter_DF.isnull().sum()

supplier_id                  0
TotalNoOfVehiclesSold        0
NoOfDefaulted_Sales        287
Defaults%_DueToSupplier    287
dtype: int64

In [204]:
Train_ManipCompleted_3 = pd.merge(Train_ManipCompleted_1,Supplier_Defaulter_DF,how ='left', left_on='supplier_id',right_on='supplier_id')

In [205]:
Train_ManipCompleted_3.columns

Index(['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',

In [207]:
Train_ManipCompleted_3.head()

,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Age,HowManyDaysSinceDisburse,State_ID,...,Bureau_score_description_No Bureau History Available,Bureau_score_description_Not Scored: More than 50 active Accounts found,Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive),Bureau_score_description_Not Scored: No Updates available in last 36 months,Bureau_score_description_Not Scored: Not Enough Info available on the customer,Bureau_score_description_Not Scored: Only a Guarantor,Bureau_score_description_Not Scored: Sufficient History Not Available,TotalNoOfVehiclesSold,NoOfDefaulted_Sales,Defaults%_DueToSupplier
0,50578,58400,89.55,67,22807,45,1441,35,403,6,...,1,0,0,0,0,0,0,55,20.0,36.363636
1,47145,65550,73.23,67,22807,45,1502,33,201,6,...,0,0,0,0,0,0,0,55,20.0,36.363636
2,53278,61360,89.63,67,22807,45,1497,33,462,6,...,1,0,0,0,0,0,0,55,20.0,36.363636
3,57513,66113,88.48,67,22807,45,1501,25,171,6,...,0,0,0,0,0,0,0,55,20.0,36.363636
4,52378,60300,88.39,67,22807,45,1495,41,201,6,...,1,0,0,0,0,0,0,55,20.0,36.363636


In [209]:
Train_ManipCompleted_3 = Train_ManipCompleted_3[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES', 'loan_default',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available','Defaults%_DueToSupplier']]

In [210]:
IndeVectors_3 = Train_ManipCompleted_3[['disbursed_amount', 'asset_cost', 'ltv', 'branch_id', 'supplier_id',
       'manufacturer_id', 'Current_pincode_ID', 'Age',
       'HowManyDaysSinceDisburse', 'State_ID', 'Employee_code_ID',
       'MobileNo_Avl_Flag', 'Aadhar_flag', 'PAN_flag', 'VoterID_flag',
       'Driving_flag', 'Passport_flag', 'PERFORM_CNS.SCORE', 'PRI.NO.OF.ACCTS',
       'PRI.ACTIVE.ACCTS', 'PRI.OVERDUE.ACCTS', 'PRI.CURRENT.BALANCE',
       'PRI.SANCTIONED.AMOUNT', 'PRI.DISBURSED.AMOUNT', 'SEC.NO.OF.ACCTS',
       'SEC.ACTIVE.ACCTS', 'SEC.OVERDUE.ACCTS', 'SEC.CURRENT.BALANCE',
       'SEC.SANCTIONED.AMOUNT', 'SEC.DISBURSED.AMOUNT', 'PRIMARY.INSTAL.AMT',
       'SEC.INSTAL.AMT', 'NEW.ACCTS.IN.LAST.SIX.MONTHS',
       'DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS', 'AvgAcctAgeInMonths',
       'CredHistLenInMonts', 'NO.OF_INQUIRIES',
       'EmploymentType_Salaried', 'EmploymentType_Self employed',
       'Bureau_score_description_A-Very Low Risk',
       'Bureau_score_description_B-Very Low Risk',
       'Bureau_score_description_C-Very Low Risk',
       'Bureau_score_description_D-Very Low Risk',
       'Bureau_score_description_E-Low Risk',
       'Bureau_score_description_F-Low Risk',
       'Bureau_score_description_G-Low Risk',
       'Bureau_score_description_H-Medium Risk',
       'Bureau_score_description_I-Medium Risk',
       'Bureau_score_description_J-High Risk',
       'Bureau_score_description_K-High Risk',
       'Bureau_score_description_L-Very High Risk',
       'Bureau_score_description_M-Very High Risk',
       'Bureau_score_description_No Bureau History Available',
       'Bureau_score_description_Not Scored: More than 50 active Accounts found',
       'Bureau_score_description_Not Scored: No Activity seen on the customer (Inactive)',
       'Bureau_score_description_Not Scored: No Updates available in last 36 months',
       'Bureau_score_description_Not Scored: Not Enough Info available on the customer',
       'Bureau_score_description_Not Scored: Only a Guarantor',
       'Bureau_score_description_Not Scored: Sufficient History Not Available','Defaults%_DueToSupplier']]
Labels_3 =  Train_ManipCompleted_3['loan_default']

In [216]:
Train_ManipCompleted_3.isnull().sum()

disbursed_amount                                                                       0
asset_cost                                                                             0
ltv                                                                                    0
branch_id                                                                              0
supplier_id                                                                            0
manufacturer_id                                                                        0
Current_pincode_ID                                                                     0
Age                                                                                    0
HowManyDaysSinceDisburse                                                               0
State_ID                                                                               0
Employee_code_ID                                                                       0
MobileNo_Avl_Flag    

In [211]:
train3_x,valid3_x,train3_y,valid3_y = model_selection.train_test_split(IndeVectors_3,Labels_3, random_state = 1234, test_size = 0.10)

In [212]:
print(len(train3_x))
print(len(train3_y))
print(len(valid3_x))
print(len(valid3_y))

209838
209838
23316
23316


In [214]:
rf_model_fe_2 = RandomForestClassifier(n_estimators=300,bootstrap=False,max_features=60)
rf_model_fe_2.fit(train3_x,list(train3_y))

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').